In [1]:
from transformers import AutoModel, BertForSequenceClassification

In [2]:
mbert_model = AutoModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
import sys
sys.path.append('./..')

In [2]:
from model.dataloader import DataModule
from model.model import FineTuner

In [3]:
dm = DataModule(
    train_path='../../data/train_new.csv',
    val_path='../../data/val_new.csv',
    tokenizer_name_or_path='bert-base-multilingual-cased',
    train_batch_size=2,
    val_batch_size=2,
    stage='fit'
)

In [6]:
dm.setup(stage='fit')

In [11]:
for id, batch in enumerate(dm.val_dataloader()):
    sample_batch = batch
    if id > 5:
        break

In [12]:
sample_batch

{'input_ids': tensor([[  101, 18939, 60965,  ...,     0,     0,     0],
         [  101, 10124, 10371,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])}

In [6]:
output = bert_model(sample_batch['input_ids'], attention_mask=sample_batch['attention_mask'])

NameError: name 'bert_model' is not defined

In [7]:
output.pooler_output.shape

NameError: name 'output' is not defined

In [4]:
bert_model = FineTuner(
    model_name_or_path='bert-base-multilingual-cased',
    learning_rate=2e-4,
    tokenizer=dm.tokenizer,
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
bert_model._step(sample_batch)

(tensor(0.7449, grad_fn=<BinaryCrossEntropyBackward>),
 tensor([[0.5715, 0.4379, 0.5243, 0.4908, 0.4935, 0.3759, 0.6350, 0.4501, 0.5480,
          0.5087],
         [0.5274, 0.4450, 0.5273, 0.5123, 0.5386, 0.4373, 0.5112, 0.5497, 0.5396,
          0.4467]], grad_fn=<SigmoidBackward>))

In [7]:
outputs = []
for id, batch in enumerate(dm.val_dataloader()):
    outputs.append(bert_model.validation_step(batch, id))
    if id > 5:
        break

/home2/shivprasad.sagare/miniconda3/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


In [9]:
outputs

[{'val_loss': tensor(0.7009, grad_fn=<BinaryCrossEntropyBackward>),
  'outputs': tensor([[0.5228, 0.5269, 0.4807, 0.4557, 0.5121, 0.4515, 0.4721, 0.4995, 0.5145,
           0.4949],
          [0.5338, 0.5524, 0.4871, 0.4286, 0.5025, 0.4551, 0.4825, 0.4923, 0.5098,
           0.4970]], grad_fn=<SigmoidBackward>),
  'labels': tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])},
 {'val_loss': tensor(0.6888, grad_fn=<BinaryCrossEntropyBackward>),
  'outputs': tensor([[0.5319, 0.5664, 0.4816, 0.4369, 0.4706, 0.4282, 0.4651, 0.5074, 0.5291,
           0.4790],
          [0.5418, 0.5513, 0.4766, 0.4247, 0.4869, 0.3938, 0.4699, 0.4951, 0.5338,
           0.4888]], grad_fn=<SigmoidBackward>),
  'labels': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])},
 {'val_loss': tensor(0.6723, grad_fn=<BinaryCrossEntropyBackward>),
  'outputs': tensor([[0.5473, 0.5553, 0.4651, 0.4227, 0.4888, 0.4017, 

In [9]:
len(outputs)

7

In [14]:
import numpy as np
from sklearn.metrics import f1_score

In [22]:
def validation_epoch_end(outputs):
        # calculate metric score here
        predictions = [x['outputs'] for x in outputs]
        print(predictions)
        predictions = np.array([np.argmax(preds.detach().cpu().numpy(), axis=1) for preds in predictions]).flatten()
        labels = [x['labels'] for x in outputs]
        print(labels)
        labels = np.array([np.argmax(labels.detach().cpu().numpy(), axis=1) for labels in labels]).flatten()
        print(predictions)
        print(labels)
        if 0 in labels:
            f1 = 0
        else:
            f1 = f1_score(labels, predictions, average='weighted')
        return f1

In [17]:
f1

NameError: name 'f1' is not defined

In [11]:
import numpy as np

In [12]:
predictions = [x['outputs'] for x in outputs]
predictions = np.array([np.argmax(preds.detach().numpy(), axis=1) for preds in predictions]).flatten()

In [14]:
predictions.shape

(24,)

In [17]:
labels = [x['labels'] for x in outputs]
labels = np.array([np.argmax(labels.detach().numpy(), axis=1) for labels in labels]).flatten()

In [18]:
labels

array([5, 8, 8, 5, 7, 6, 6, 6, 7, 3, 7, 7, 4, 7, 7, 6, 8, 4, 6, 5, 6, 3,
       6, 8])

In [20]:
from sklearn.metrics import f1_score

In [21]:
f1_score(labels, predictions, average='weighted')

0.0

In [23]:
f1_score([1, 3, 1], [1, 2, 1], average='weighted')

0.6666666666666666

In [24]:
bert_model.validation_epoch_end(outputs)

/home2/shivprasad.sagare/miniconda3/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."
